# Week 7

In [9]:
#load libraries
import os
import numpy as np
import pandas as pd

import geopandas as gpd
import xarray as xr
import rioxarray as rioxr

from shapely import Point
import matplotlib.pyplot as plt

In [4]:
#load in lulc table
lulc_fp = os.path.join(os.getcwd(),'data','land_cover.tif')
lulc = rioxr.open_rasterio(lulc_fp)
lulc

<xarray.DataArray (band: 1, y: 3000, x: 3000)>
[9000000 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.031e+06 -2.031e+06 ... -1.941e+06 -1.941e+06
  * y            (y) float64 1.762e+06 1.762e+06 ... 1.672e+06 1.672e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  2 (pixels/inch)
    TIFFTAG_SOFTWARE:        ERDAS IMAGINE
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    scale_factor:            1.0
    add_offset:              0.0

In [14]:
#load in tabular data
class_names = pd.read_csv("data/GAP_National_Terrestrial_Ecosystems.csv")
class_names

,class_label,code
0,0,0
1,South Florida Bayhead Swamp,1
2,South Florida Cypress Dome,2
3,South Florida Dwarf Cypress Savanna,3
4,South Florida Mangrove Swamp,4
...,...,...
580,"Quarries, Mines, Gravel Pits and Oil Wells",580
581,"Developed, Open Space",581
582,"Developed, Low Intensity",582
583,"Developed, Medium Intensity",583


Ways to explore raster:
- look at crs
- plot
- look at bands
- look at type

In [20]:
#look at type of lulc
type(lulc)

xarray.core.dataarray.DataArray

In [22]:
# print shape and data type
print('shape: ', lulc.shape)
print('data type: ', lulc.dtype, '\n')

shape:  (1, 3000, 3000)
data type:  uint16 



In [24]:
# check geospatial attributes
print('shape: ', lulc.shape) #shape
print('data type: ', lulc.dtype) #data type
print('# bands: ', lulc.rio.count) #number of bands
print('CRS: ', lulc.rio.crs) #CRS



shape:  (1, 3000, 3000)
data type:  uint16
# bands:  1
CRS:  EPSG:5070


In [25]:
#look at dimensions and coordinates
print(lulc.dims, lulc.coords,'\n')

('band', 'y', 'x') Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.031e+06 -2.031e+06 ... -1.941e+06 -1.941e+06
  * y            (y) float64 1.762e+06 1.762e+06 ... 1.672e+06 1.672e+06
    spatial_ref  int64 0 



In [26]:
#use squeeze function to get rid of dimensions that are of length 1
lulc = lulc.squeeze()
print(lulc.dims, lulc.coords,'\n')

('y', 'x') Coordinates:
    band         int64 1
  * x            (x) float64 -2.031e+06 -2.031e+06 ... -1.941e+06 -1.941e+06
  * y            (y) float64 1.762e+06 1.762e+06 ... 1.672e+06 1.672e+06
    spatial_ref  int64 0 



In [27]:
# remove coordinates associated to band
lulc = lulc.drop('band')
print(lulc.dims, lulc.coords)

('y', 'x') Coordinates:
  * x            (x) float64 -2.031e+06 -2.031e+06 ... -1.941e+06 -1.941e+06
  * y            (y) float64 1.762e+06 1.762e+06 ... 1.672e+06 1.672e+06
    spatial_ref  int64 0


In [41]:
#use np.unique to see how many pixels there are
value, counts = np.unique(lulc, return_counts = True)
value, counts

(array([ 39,  42,  56, 146, 148, 159, 162, 163, 164, 174, 175, 176, 179,
        181, 183, 184, 187, 266, 277, 278, 282, 297, 301, 304, 360, 443,
        457, 459, 470, 471, 472, 476, 477, 489, 490, 497, 498, 533, 534,
        539, 540, 541, 546, 547, 550, 552, 553, 556, 557, 558, 567, 568,
        569, 570, 573, 575, 579, 581, 582, 583, 584], dtype=uint16),
 array([ 639385,  584143,  518722,     157,  121486,  526411,   19768,
         763950,   19664,    6042,  301468,   11910,  127057,  829579,
         663763,     452,     401,    2810,    7981,  259737,   13425,
           4009,   71858,  871898,    4086,       3,       2,       7,
         606237,       2,   15757,   27919,     507,  385986,   17067,
             44,     313,       3,    1836,    2582,     572,      11,
              7,       4, 1417008,     203,    2261,   40675,    7432,
            356,     164,   10042,    1087,      55,      84,     226,
          36052,   40296,   12947,    1955,     136]))

In [42]:
#create an empty dataframe
pix_counts = pd.DataFrame

In [49]:
pix_counts = {'code' : value,
     'pixels' : counts
     }
pix_counts = pd.DataFrame(pix_counts)
pix_counts

,code,pixels
0,39,639385
1,42,584143
2,56,518722
3,146,157
4,148,121486
...,...,...
56,579,36052
57,581,40296
58,582,12947
59,583,1955


In [50]:
classes = pd.merge(pix_counts,
                   class_names, 
                   how = 'left',
                   on = 'code')
classes

,code,pixels,class_label
0,39,639385,California Central Valley Mixed Oak Savanna
1,42,584143,California Lower Montane Blue Oak-Foothill Pin...
2,56,518722,Mediterranean California Mixed Oak Woodland
3,146,157,Inter-Mountain Basins Subalpine Limber-Bristle...
4,148,121486,Rocky Mountain Aspen Forest and Woodland
...,...,...,...
56,579,36052,Open Water (Fresh)
57,581,40296,"Developed, Open Space"
58,582,12947,"Developed, Low Intensity"
59,583,1955,"Developed, Medium Intensity"
